In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

#全部資料
raw_data_cnt = 1095
used_data = []
y_train = []
raw_data = []

#讀取資料
for x in range(raw_data_cnt):
    with open('./PA1-data/'+str(x+1)+'.txt', 'r') as file:
        file_contents = file.read()
        raw_data.append(file_contents)

#讀取訓練資料
with open("./training_new.txt", 'r') as file:
    training_data = file.read()
    training_data = training_data.split('\n')
    training_data = [x.split(' ') for x in training_data]
    for x in range(len(training_data)):
        training_data[x] = training_data[x][0:-1]
        training_data[x] = [int(y) for y in training_data[x]]

#分割資料
x_train = []
for x in range(len(training_data)):
    for y in range(1, len(training_data[x])):
        x_train.append(raw_data[training_data[x][y]-1])
        y_train.append(training_data[x][0])



x_train = np.array(x_train)
y_train = np.array(y_train)


In [10]:
from keras_bert import extract_embeddings
from keras_bert import load_vocabulary
from keras_bert import Tokenizer

model_path = './uncased_L-12_H-768_A-12'
dict_path = model_path+'/vocab.txt'

bert_token_dict = load_vocabulary(dict_path)
bert_tokenizer = Tokenizer(bert_token_dict)

for x in raw_data:
    tokens = bert_tokenizer.tokenize(x)
    indices, segments = bert_tokenizer.encode(first=x)

embeddings = extract_embeddings(model_path, x_train)


/Users/yl/Library/Caches/pypoetry/virtualenvs/text_processing-dU80QEBk-py3.9/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


1/1 [==============================] - 2s 2s/step


In [11]:
print(len(embeddings))
X_convert = []

for x in range(len(embeddings)):
    X_convert.append(embeddings[x][0])

x_train, x_test, y_train, y_test = train_test_split(X_convert, y_train, test_size=0.1, random_state=6, stratify = y_train)


195


In [12]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier

# Create a one-vs-all classifier using SVM with a linear kernel
model_linear = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
model_linear.fit(x_train, y_train)

OneVsRestClassifier(estimator=SVC(kernel='linear', probability=True))

In [13]:
from sklearn.metrics import classification_report

y_pred_linear = model_linear.predict(x_test)
report_linear = classification_report(y_test, y_pred_linear, zero_division=1)

print("Linear SVM Model:")
print(report_linear)

Linear SVM Model:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [14]:
# do grid search on svm parameters 
from sklearn.model_selection import GridSearchCV

# Set the parameters by cross-validation
tuned_parameters = [{'C': [1e-1, 1e-2, 1e-3, 1e-4,1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]}]

# scores = ['precision', 'recall']
scores = ['precision']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(probability=True), tuned_parameters, scoring='%s_macro' % score)
    clf.fit(x_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")

    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']

    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

    print()
    print("Detailed classification report:")
    print()

    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")

    y_true, y_pred = y_test, clf.predict(x_test)
    print(classification_report(y_true, y_pred, zero_division=1))
    print()

# get the best model
best_model = clf.best_estimator_
best_model.fit(x_train, y_train)



# Tuning hyper-parameters for precision



/Users/yl/Library/Caches/pypoetry/virtualenvs/text_processing-dU80QEBk-py3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yl/Library/Caches/pypoetry/virtualenvs/text_processing-dU80QEBk-py3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yl/Library/Caches/pypoetry/virtualenvs/text_processing-dU80QEBk-py3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

Best parameters set found on development set:

{'C': 0.1, 'kernel': 'linear'}

Grid scores on development set:
0.008 (+/-0.015) for {'C': 0.1}
0.008 (+/-0.015) for {'C': 0.01}
0.008 (+/-0.015) for {'C': 0.001}
0.008 (+/-0.015) for {'C': 0.0001}
0.922 (+/-0.099) for {'C': 1}
0.944 (+/-0.049) for {'C': 10}
0.944 (+/-0.049) for {'C': 100}
0.944 (+/-0.049) for {'C': 1000}
0.039 (+/-0.070) for {'C': 0.001, 'kernel': 'linear'}
0.927 (+/-0.116) for {'C': 0.01, 'kernel': 'linear'}
0.959 (+/-0.043) for {'C': 0.1, 'kernel': 'linear'}
0.959 (+/-0.043) for {'C': 1, 'kernel': 'linear'}
0.959 (+/-0.043) for {'C': 10, 'kernel': 'linear'}
0.959 (+/-0.043) for {'C': 100, 'kernel': 'linear'}
0.959 (+/-0.043) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       

SVC(C=0.1, kernel='linear', probability=True)

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split

#全部資料
raw_data_cnt = 1095
used_data = []
y_train = []
raw_data = []

#讀取資料
for x in range(raw_data_cnt):
    with open('./PA1-data/'+str(x+1)+'.txt', 'r') as file:
        file_contents = file.read()
        raw_data.append(file_contents)

#讀取訓練資料
with open("./training_new.txt", 'r') as file:
    training_data = file.read()
    training_data = training_data.split('\n')
    training_data = [x.split(' ') for x in training_data]
    for x in range(len(training_data)):
        training_data[x] = training_data[x][0:-1]
        training_data[x] = [int(y) for y in training_data[x]]

#分割資料
x_train = []
for x in range(len(training_data)):
    for y in range(1, len(training_data[x])):
        x_train.append(raw_data[training_data[x][y]-1])
        y_train.append(training_data[x][0])

training_data = np.array(training_data)
training_data = np.delete(training_data, 0, 1)
training_data = training_data.flatten()
training_data.sort()

predict = np.delete(raw_data, training_data-1)
predict = extract_embeddings(model_path, predict)




/Users/yl/Library/Caches/pypoetry/virtualenvs/text_processing-dU80QEBk-py3.9/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


1/1 [==============================] - 2s 2s/step


In [16]:
real_predict = []
for x in range(len(predict)):
    real_predict.append(predict[x][0])

y_pred = best_model.predict(real_predict)
# save the prediction to a csv
import csv
with open('./prediction2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Value'])
    i = 0
    for x in range(raw_data_cnt):
        if x+1 not in training_data:
            writer.writerow([x+1, y_pred[i]])
            i+=1